<a href="https://colab.research.google.com/github/kyle-gao/GRSS_TrackMSD2021/blob/main/MakeTilesDeepGlobe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Copyright 2021 Yi Lin(Kyle) Gao
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License."""

'\nCopyright 2020 Yi Lin(Kyle) Gao\n#@title Licensed under the Apache License, Version 2.0 (the "License");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n# https://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an "AS IS" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.'

#DeepGlobe Dataset
https://www.kaggle.com/balraj98/deepglobe-land-cover-classification-dataset#__sid=js0

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import shutil
from PIL import Image
import os
from google.colab import drive
import PIL

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
shutil.unpack_archive("/content/drive/MyDrive/DeepGlobeLandCover.zip.zip",'/content/DeepGlobe')

In [ ]:
test_x = tf.keras.preprocessing.image.load_img(
    "/content/DeepGlobe/train/100694_sat.jpg", grayscale=False, color_mode='rgb', target_size=None,
    interpolation='nearest')
test_x

In [ ]:
data_dir = "/content/DeepGlobe/train"
list_ds = tf.data.Dataset.list_files(str(data_dir+"/*.png"),shuffle=False) #DO NOT SHUFFLE
#dataset is made up of strings

In [ ]:
def to_categorical(tensor,class_dict):
  """
  converts last dimension to categorical according to keys
  """
  for k,v in class_dict.items():
    tensor[tensor==k]=v
  return tensor

##Label
Each satellite image is paired with a mask image for land cover annotation. The mask is a RGB image with 7 classes of labels, using color-coding (R, G, B) as follows.

Urban land: 0,255,255 - Man-made, built up areas with human artifacts (can ignore roads for now which is hard to label)

Agriculture land: 255,255,0 - Farms, any planned (i.e. regular) plantation, cropland, orchards, vineyards, nurseries, and ornamental horticultural areas; confined feeding operations.

Rangeland: 255,0,255 - Any non-forest, non-farm, green land, grass

Forest land: 0,255,0 - Any land with x% tree crown density plus clearcuts.

Water: 0,0,255 - Rivers, oceans, lakes, wetland, ponds.

Barren land: 255,255,255 - Mountain, land, rock, dessert, beach, no vegetation

Unknown: 0,0,0 - Clouds and others

File names for satellite images and the corresponding mask image are id _sat.jpg and id _mask.png. id is a randomized integer.

Please note:

The values of the mask image may not be pure 0 and 255. When converting to labels, please binarize them at threshold 128.|

In [ ]:
size = (512,512)
filenames = list(list_ds)
padding = 'VALID'


def deepglobe_write_tiles(filenames,size=(512,512),padding='VALID',save_dir = "/content/DeepGlobe224/"):
  """
  Args-
  filenames: tensorflow list_files dataset object
  size: tuple of ints
  padding=one of "VALID" "SAME"
  save_dir-save directory
  """
  (h,w) = size
  for f in filenames:
    fn=tf.strings.split(f,"_")
    image_fn = (fn[0]+"_sat.jpg").numpy()
    label_fn = (fn[0]+"_mask.png").numpy()

    image = tf.keras.preprocessing.image.load_img(image_fn)
    image = tf.keras.preprocessing.image.img_to_array(image)

    label = tf.keras.preprocessing.image.load_img(label_fn)
    label = tf.keras.preprocessing.image.img_to_array(label)
    #(H,W,3)

    """ - do this step in preprocessing instead since the encoding rescales everything to 255
    #binarization
    label [label >= 128] = 255
    label[label < 128] = 0

    labelnew = label[:,:,0]+0.1*label[:,:,1]+0.01*label[:,:,2] #contracts the last dimension without losing class information

    class_dict = {(25.5+2.55):0, (255+25.5):1, (255+2.55):2,(25.5):3,2.55:4,(255+25.5+2.55):5,0:6}
    labelnew = to_categorical(labelnew,class_dict) #(H,W) """
    
    
    image = tf.expand_dims(image,axis=0)
    image_tiles =  tf.image.extract_patches(images=image,
                           sizes=[1,h, w, 1],
                           strides=[1,h, w, 1],
                           rates=[1, 1, 1, 1],
                           padding=padding)
    image_tiles = tf.reshape(image_tiles, [-1,h,w,3])
  
    #label = tf.expand_dims(labelnew,axis=-1)
    label = tf.expand_dims(label,axis=0)
    label_tiles =  tf.image.extract_patches(images=label,
                           sizes=[1,h, w, 1],
                           strides=[1,h, w, 1],
                           rates=[1, 1, 1, 1],
                           padding=padding)
    label_tiles = tf.reshape(label_tiles, [-1,h,w,3])

    if not(os.path.isdir(save_dir)):
      os.mkdir(save_dir)


    for i in range(label_tiles.shape[0]):
      imgtile_fn = (save_dir+fn[0].numpy().decode("utf-8").split("/")[-1]+"_"+str(i)+"_sat.jpg")
      labeltile_fn = (save_dir+fn[0].numpy().decode("utf-8").split("/")[-1]+"_"+str(i)+"_mask.png")

      tf.keras.preprocessing.image.save_img(imgtile_fn,image_tiles[i,:,:,:])
      tf.keras.preprocessing.image.save_img(labeltile_fn,label_tiles[i,:,:,:])



    print(image_fn)

In [ ]:
deepglobe_write_tiles(filenames)

b'/content/DeepGlobe/train/100694_sat.jpg'
b'/content/DeepGlobe/train/102122_sat.jpg'
b'/content/DeepGlobe/train/10233_sat.jpg'
b'/content/DeepGlobe/train/103665_sat.jpg'
b'/content/DeepGlobe/train/103730_sat.jpg'
b'/content/DeepGlobe/train/104113_sat.jpg'
b'/content/DeepGlobe/train/10452_sat.jpg'
b'/content/DeepGlobe/train/10901_sat.jpg'
b'/content/DeepGlobe/train/111335_sat.jpg'
b'/content/DeepGlobe/train/114433_sat.jpg'
b'/content/DeepGlobe/train/114473_sat.jpg'
b'/content/DeepGlobe/train/114577_sat.jpg'
b'/content/DeepGlobe/train/115444_sat.jpg'
b'/content/DeepGlobe/train/119012_sat.jpg'
b'/content/DeepGlobe/train/119079_sat.jpg'
b'/content/DeepGlobe/train/119_sat.jpg'
b'/content/DeepGlobe/train/120245_sat.jpg'
b'/content/DeepGlobe/train/120625_sat.jpg'
b'/content/DeepGlobe/train/122104_sat.jpg'
b'/content/DeepGlobe/train/122178_sat.jpg'
b'/content/DeepGlobe/train/123172_sat.jpg'
b'/content/DeepGlobe/train/124529_sat.jpg'
b'/content/DeepGlobe/train/125510_sat.jpg'
b'/content/DeepGl

In [ ]:
shutil.make_archive("/content/DeepGlobe_512",'zip',"/content/DeepGlobe224")
shutil.copy2("/content/DeepGlobe_512.zip","/content/drive/MyDrive")

'/content/drive/MyDrive/DeepGlobe_512.zip'